In [3]:
import cv2
import numpy as np
import math
import cv2.bgsegm
import csv
import os

from skimage import img_as_ubyte, filters
from skimage.morphology import closing, square, remove_small_objects #, flood_fill
from enum import Enum

In [4]:
class ProcessingType(Enum):
    DENOISE = 1
    MOG = 2
    TEMPORAL_MEDIAN = 3
    COMBINED = 4

In [5]:
video_path = "../video/rilevamento-intrusioni-video.wm"
cap = cv2.VideoCapture(video_path)

assert cap.isOpened(), "Not opened!"

fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
length = total_frame_count / fps

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f"[I] Video FPS: {fps}")
print(f"[I] Video Total frame count: {total_frame_count}")
print(f"[I] Video Length: {length}")
print(f"[I] Video Frame Width: {width}")
print(f"[I] Video Frame height: {height}")

[I] Video FPS: 12
[I] Video Total frame count: 503
[I] Video Length: 41.916666666666664
[I] Video Frame Width: 320
[I] Video Frame height: 240


In [6]:
def preprocessing(frame, type):
    """Apply all the preprocessing steps to a copy of the passed ``frame``.

    Applies a series of linear and non-linear filters base on the ``type`` passed.

    Parameters
    ----------
    frame : ndarray
        Grayscale input image.
    type : ProcessingType
        Instance of ProcessingType, it indicates which type of preprocessing will be applied.
        - ``ProcessingType.DENOISE``

    Returns
    -------
    processed : ndarray
        A processed copy grayscale image.
    """
    
    assert isinstance(type, ProcessingType), "type must be an instance of ProcessingType (Enum)"
    output = frame.copy()
    
    if type == ProcessingType.DENOISE:
        # output = cv2.medianBlur(output, 5)
        # output = cv2.GaussianBlur(output, (5, 5), 2)
        output = cv2.bilateralFilter(output, 3, 75, 75)

    else:
        raise ValueError("ProcessingType.type not found")
    
    return output

In [7]:
def create_mask_from_approxPoly(frame, epsilon = 1.0):
    """Use approxPoly() to redraw the contour with less vertices.

    Parameters
    ----------
    frame : ndarray
        Foreground mask input image.
    epsilon : float
        Approximation accuracy. This is the maximum distance between the original curve and its approximation

    Returns
    -------
    processed : ndarray
        A processed copy the input image.
    """
    
    contours, _ = cv2.findContours(frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contours_poly = [cv2.approxPolyDP(contour, epsilon, True) for contour in contours]
    mask = np.zeros(frame.shape, np.uint8)
    approx_poly = cv2.drawContours(mask, contours_poly, -1, (255,255,255), cv2.FILLED)
    
    return approx_poly

In [8]:
def postprocessing(fgmask, type):
    """Apply, to a copy of the passed ``fgmask``, a pipeline of filters and morphological operators to improve the segmentation.

    Parameters
    ----------
    fgmask : ndarray
        Foreground mask input image.
    type : ProcessingType
        Instance of ProcessingType, it indicates which type of preprocessing will be applied.
        - ``ProcessingType.MOG``
        - ``ProcessingType.TEMPORAL_MEDIAN``
        - ``ProcessingType.COMBINED``

    Returns
    -------
    processed : ndarray
        A processed copy the input image.
    """
    assert isinstance(type, ProcessingType), "type must be an instance of ProcessingType (Enum)"
    output = fgmask.copy()
    element = np.ones((3,3), np.uint8)

    if type == ProcessingType.MOG:
        output = create_mask_from_approxPoly(output)
        output = cv2.medianBlur(output, 3)
        output = create_mask_from_approxPoly(output)
        output = cv2.dilate(output, element, iterations=1)
        output = cv2.morphologyEx(output, cv2.MORPH_CLOSE, element, iterations=1)        
        # output = flood_fill(output, (1,1), 127, connectivity=25)
        output = create_mask_from_approxPoly(output)

    elif type == ProcessingType.TEMPORAL_MEDIAN:
        output = create_mask_from_approxPoly(output)
        output = cv2.medianBlur(output, 5)
        output = create_mask_from_approxPoly(output)
        output = cv2.morphologyEx(output, cv2.MORPH_CLOSE, element, iterations=2)
        output = create_mask_from_approxPoly(output)
        # output = cv2.medianBlur(output, 3)

    elif type == ProcessingType.COMBINED:
        element = np.ones((5,5), np.uint8)
        output = cv2.dilate(output, element, iterations=1)
        output = cv2.morphologyEx(output, cv2.MORPH_CLOSE, element, iterations=2)

    else:
        raise ValueError("ProcessingType.type not found")

    return output

In [9]:
def compute_learning_rate(frame_count, history_length, dynamic = True, bias = 0):
    """ Computes the learning rate.

    Parameters
    ----------
    frame_count : int 
        Current frame number
    history_lenght : int
        Number of frames in buffer
    dynamic : boolean
    bias : float
        Constant added to the computed learning rate
    
    Returns
    -------
    learning_rate : float
        the learning rate to use

    """

    # opencv/opencv/modules/video/src/bgfg_gaussmix2.cpp  #871
    # 1./std::min( 2*nframes, history )

    assert frame_count > 0, "Frame Count must be greater than zero"
    if dynamic and frame_count < history_length:
        return (1 / frame_count) + bias
    

    # const double alpha1 = 1.0f - learningRate;
    # float weight = alpha1*gmm[mode].weight + prune; //need only weight if fit is found

    return (1 / history_length) + bias

In [10]:
def background_subtraction(frame, background, adaptive, args):
    """ Computes the absolute difference between the frame and the background then applies a threshold to segment the
    background and the foreground.
    
    Parameters
    ----------
    frame : ndarray
        image, current frame
        
    background : ndarray
        image, estimated reference frame

    adaptive : boolean
        ``True``: use adaptive thresholding, ``False`` use hysteresis thresholding
    
    args : Tuple
        It contains two types of arguments based on the ``adaptive`` flag\\
        If ``adaptive``:
        - C : float
            Constant subtracted from the mean or weighted mean
        - block_size : int
            Neighbourhood size
        
        else:
        - threshold_low : int 
        - threshold_high : int
            
    Returns
    -------
    fgmask : ndarray
        binary image with the foreground white (255).
    """
    diff = cv2.absdiff(frame, background)

    if adaptive:
        (C, block_size) = args
        fgmask = cv2.adaptiveThreshold(diff, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, block_size, C)
    else:
        (t_low, t_high) = args
        hysteresis = filters.apply_hysteresis_threshold(diff, t_low, t_high)
        fgmask = img_as_ubyte(hysteresis)

        # fgmask = combine_fgmask_with_edges(fgmask, diff_edges)

    return fgmask
        

In [11]:
def get_roi_foreground(fgmask):
    """ Computes the absolute a rectangular ROI containing the foreground mask.
    
    Parameters
    ----------
    fgmask : ndarray
        foreground mask
        
    Returns
    -------
    result : Tuple
    - found : boolean
    - coordinates : Tuple
        (x_min, x_max, y_min, y_max)
    """

    # many dilate and closing to create a bigger figure for the person and make it easier to compute the roi
    kernel = np.ones((7,7),np.uint8)
    dilate = cv2.dilate(fgmask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5)), iterations=1)
    bw = cv2.morphologyEx(dilate, cv2.MORPH_CLOSE, kernel, iterations=1)
    #erode = cv2.erode(bw.astype(np.uint8) * 255, None, iterations=3)
    erode = cv2.medianBlur(bw, 3)
    dilated = cv2.dilate(erode, None, iterations=3)

    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    real_countours = []
    for contour in contours:
        # (x, y, w, h) = cv2.boundingRect(contour)
        if cv2.contourArea(contour) > 300:
            real_countours.append(contour)
        
    
    if len(real_countours) == 0:
        return False, None
    
    # Define the ROI 
    x_min = min([np.min(cnt[:, :, 0]) for cnt in real_countours])
    x_max = max([np.max(cnt[:, :, 0]) for cnt in real_countours])
    y_min = min([np.min(cnt[:, :, 1]) for cnt in real_countours])
    y_max = max([np.max(cnt[:, :, 1]) for cnt in real_countours])

    padding = 20
    x_min = max(x_min - padding, 0)
    x_max = min(x_max + padding, width)
    y_min = max(y_min - padding, 0)
    y_max = min(y_max + padding, height)

    return True, (x_min, x_max, y_min, y_max)

In [12]:
def diff_edges(frame1, frame2, isFrame1Binary = False, isFrame2Binary = False):
    """Computes the difference of the egdes of the frame1 and egdes of the frame2.
    If the frame is binary, it will compute the edges using the morphological operator erode.
    Instead, if the frame is grayscale, it will compute the edges using the Canny algorithm.
    It will remove from frame1 the edges also founded in frame2.

    Parameters
    ----------
    frame1 : ndarray
    frame2 : ndarray
    isFrame1Binary : boolean
        ``True``: Frame 1 is binary
    isFrame2Binary : boolean
        ``True``: Frame 2 is binary


    Returns
    ----------
    img : ndarray
        binary image with differences of edges
    """
    upthresh = 150
    lothresh = upthresh // 2

    # frame2 = preprocessing(frame2, ProcessingType.EDGES)
    if not isFrame2Binary:
        edge_frame2 = cv2.Canny(frame2, lothresh, upthresh)
    else:
        diff_eroded = cv2.erode(frame2, None, iterations=1)
        edge_frame2 =  frame2 - diff_eroded

    # enlarge frame2 edges to better cancel out with the ones of the frame1
    # elem = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    # edge_background = cv2.dilate(edge_background, elem, iterations=1)
    
    # frame1 = preprocessing(frame1, ProcessingType.EDGES)
    if not isFrame1Binary:
        edge_frame1 = cv2.Canny(frame1, lothresh, upthresh)
    else:
        diff_eroded = cv2.erode(frame1, None, iterations=1)
        edge_frame1 =  frame1 - diff_eroded
    
    # edge_diff will contain the edges that are present in the frame1 but not in the frame2
    # frame1  frame2       output
    #      0       0  -->       0
    #    255       0  -->     255
    #      0     255  -->       0
    #    255     255  -->       0
    edge_diff = edge_frame1 - np.min([edge_frame1, edge_frame2], axis=0)

    processed = remove_small_objects(edge_diff.astype(bool), min_size=15, connectivity=2).astype(int)

    # black out pixels
    mask_x, mask_y = np.where(processed == 0)
    edge_diff[mask_x, mask_y] = 0
    
    return edge_diff


In [13]:
def imshow_components(labels_diff, labels_comb):
    """Color the different blobs found by the ConnectedComponents Analysis.
    This method is NOT general purpose and it will color the blobs according to the groundtruth colors.
    - red : person
    - green : removed book
    - blue : added book

    Parameters
    ----------
    labels_diff : ndarray
    labels_comb : ndarray

    Returns
    ----------
    img : ndarray
        binary image with differences of edges
    """
        
    # We separate the labeling for static objects (books) and dynamic objects (person)
    # to maintain color consistency for all elements across different frames.
    colors_statics = [(0,255,0), (255,0,0), (80,80,80), (0, 233, 0), (0,0,255)]
    used_statics = [False, False, False, False, False] # Track used colors for static objects
    
    colors_dynamics = [(0,0,255), (0,255,0), (255,0,0), (254,0,0), (0, 80, 0)]
    used_dynamics = [False, False, False, False, False] # Track used colors for dynamic objects

    unique_labels_diff = np.unique(labels_diff)
    unique_labels_comb = np.unique(labels_comb)

    # Create an output image initialized to black (empty background)
    labeled_img = np.zeros((*labels_comb.shape, 3), dtype=np.uint8)
    color_index = 0

    # labeling the static elements
    for label in unique_labels_diff:
        if label == 0:
            continue  # Ignore the background
        labeled_img[labels_diff == label] = colors_statics[color_index]
        used_statics[color_index] = True # used to avoid using the same color for different elements
        color_index += 1
    
    # labeling the dynamic elements
    for label in unique_labels_comb:
        if label == 0:
            continue  # Ignore the background

        # Find available colors that haven't been used for dynamic objects
        free_indexes = [i for i in range(len(used_dynamics)) if used_dynamics[i] == False]
        color_index = free_indexes[0]

        # Ensure dynamic objects do not reuse colors assigned to static objects
        for pos, i in enumerate(free_indexes):
            found = False
            for j in range(len(colors_statics)):
                if colors_statics[j] == colors_dynamics[i] and used_statics[j] == False:
                    color_index = i
                    found = True
                    break
                elif colors_statics[j] == colors_dynamics[i] and used_statics[j] == True:
                    color_index = free_indexes[pos+1]
            if found == True:
                break

        # Apply the selected color to the corresponding region
        labeled_img[labels_comb == label] = colors_dynamics[color_index]
        used_dynamics[color_index] = True
  
    return labeled_img

In [14]:
def process_contours(input_mask, min_area = 200):
    """ Process the contours found in the input mask.
    It will remove the contours with an area less than the min_area.

    Parameters
    ----------
    input_mask : ndarray
        binary image with the foreground white.
    min_area : int
        minimum area to consider a contour as valid
    
    Returns
    -------
    filled_mask : ndarray
        binary image with the filled contours    
    """

    mask = np.zeros(input_mask.shape, np.uint8)
    contours, _ = cv2.findContours(input_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    real_countours = []
    
    for contour in contours:
        if cv2.contourArea(contour) > min_area:
            real_countours.append(contour)
    
    filled_mask = cv2.drawContours(mask, real_countours, -1, (255,255,255), cv2.FILLED)

    return filled_mask

In [15]:
def find_phantoms(contours, frame, background, diff):
    """ Find the false objects in the scene.
    It will compare the edges of the contours with the edges of the background.
    If the edges of the contours are not present in the background, it will be considered a phantom.

    Parameters
    ----------
    contours : ndarray
        contours found in the scene
    frame : ndarray
        current frame
    background : ndarray
        background frame
    diff : ndarray
        mask of the long-term elements of the scene

    Returns
    -------
    phantom : boolean
        True if the object is a phantom, False otherwise
    """

    # Compute the bounding box around the contour
    rect = cv2.minAreaRect(contours)
    box = cv2.boxPoints(rect)
    box = np.int32(box)
    box = (box[0], box[2])

    # Calculate the minimum and maximum x and y coordinates for the region of interest (ROI) with padding.
    padding = 5
    x_min = max(min(box[0][0], box[1][0]) - padding, 0)
    x_max = min(max(box[0][0], box[1][0]) + padding, width)
    y_min = max(min(box[0][1], box[1][1]) - padding, 0)
    y_max = min(max(box[0][1], box[1][1]) + padding, height)

    # Extract the relevant region from the frame and background based on the bounding box and compute the Canny edge differences between the frame and background 
    # in the selected ROI, then count the number of edge pixels (white pixels).
    presence_in_frame = diff_edges(frame[y_min:y_max, x_min:x_max], background[y_min:y_max, x_min:x_max])
    presence_in_background_1 = diff_edges(background[y_min:y_max, x_min:x_max], frame[y_min:y_max, x_min:x_max])
    count_presence_in_frame = np.sum(presence_in_frame == 255)
    count_presence_in_background_1 = np.sum(presence_in_background_1 == 255)

    # Looking at a long term window, the background will change and absorb those elements that where inserted/removed. For that reason, the previous method will not work
    # anymore. We have computed the mask containing the long-term elements of the scene, so we will use it to compare the edges of the contours with the edges of the background, 
    # like that it will be possible for us to continue to detect "phantom" objects.
     
    # Extract the relevant region from the diff image and compute the edge differences between the diff image and the background in the selected ROI,
    # then count the number of edge pixels (white pixels).
    presence_in_diff = diff_edges(diff[y_min:y_max, x_min:x_max], background[y_min:y_max, x_min:x_max], isFrame1Binary=True)
    presence_in_background_2 = diff_edges(background[y_min:y_max, x_min:x_max], diff[y_min:y_max, x_min:x_max], isFrame2Binary=True)
    count_presence_in_diff = np.sum(presence_in_diff == 255)
    count_presence_in_background_2 = np.sum(presence_in_background_2 == 255)

    # If no edges are found in both images, return -1 to indicate no significant difference.
    if count_presence_in_frame == 0 and count_presence_in_background_1 == 0 and count_presence_in_diff == 0 and count_presence_in_background_2 == 0:
        return -1

    # Determine whether the object is a "phantom" (false object) based on the comparison of edge counts.   
    # First condition: shorter time window
    # Second condition: longer time window   
    phantom = (count_presence_in_frame <= count_presence_in_background_1) or (count_presence_in_diff >= count_presence_in_background_2)

    return phantom

In [16]:
def test_background_ratio(background_ratio, history, learning_rate = -1):
    """ Calculates the number of frames required for a static object to be considered part of the background """
    return math.log(background_ratio) / math.log(1 - 1/history)

# Main

In [17]:
# -------------------------------
# |           CONFIG            |
# -------------------------------
BLACK = 0
SHOW_GUI = False
font = cv2.FONT_HERSHEY_SIMPLEX
file_output = 'output.csv'

# -------------------------------
# |      HYPERPARAMETERS        |
# -------------------------------
FRAME_BUFFERED_PER_SECOND = 2                   # 2 images added each second to frame buffer
MAX_HISTORY = 3 * fps                           # 3x12 frames stored in the "circular buffer"
SKIP_FRAMES = fps // FRAME_BUFFERED_PER_SECOND

STATIC_THRESHOLD = 40                           # used in the absolute difference
STATIC_THRESHOLD_HYSTERESIS = 30

LEARNING_PHASE = 5 * fps                        # Initialization phase to estimate the reference frame


In [18]:
def create_mog2_background_subtractor():
    mog2 = cv2.createBackgroundSubtractorMOG2()

    # [ HISTORY ]
    # Set's the number of last frames that affect the background model.
    mog2.setHistory(LEARNING_PHASE)

    # [ BACKGROUND RATIO ] 
    # Determines the portion of the history used to model the background, directly influencing how quickly new objects are integrated into it.
    # A higher backgroundRatio results in a more stable model that is less sensitive to temporary changes, while a lower backgroundRatio allows for faster adaptation to new objects. 
    # If a foreground pixel maintains a nearly constant value for approximately backgroundRatio × history frames, it is reclassified as background and incorporated into the model as a 
    # new Gaussian component.
    # cf is a measure of the maximum portion of the data that can belong to foreground objects without inﬂuencing the background model. 
    # If the object remains static long enough, its weight becomes larger than cf and it can be considered to be part of the background.
    # For example, the choice of cf​=0.1 implies that the algorithm will retain 90% of the existing background model, making it more stable. 
    # (cf=0.1 --> background_ratio=0.9)
    # A lower cf​ (higher Background Ratio) will make the model more stable, while a higher cf​ (lower Background Ratio) will allow for quicker adaptation.
    mog2.setBackgroundRatio(70 / 100)             # cf=0.3 --> background_ratio=0.7
    frames_of_stability = test_background_ratio(mog2.getBackgroundRatio(), mog2.getHistory())
    print(f"A new object should remain static for {frames_of_stability:.{3}} frames i.e. {(frames_of_stability / fps):.{3}} seconds")

    # [ SHADOWS ]
    mog2.setDetectShadows(True)
    mog2.setShadowThreshold(0.70)                 # A lower value may help in detecting more shadows.
    mog2.setShadowValue(BLACK)                    # Detect shadows and hide them

    # [ VARIANCES ]
    # mog2.setVarInit(25)                         # default: 15
    # mog2.setVarMax(5 * 25)                      # default: 5 * 15
    # mog2.setVarMin(4)                           # default: 4

    # [ THRESHOLDS ]
    # The main threshold on the squared Mahalanobis distance to decide if the sample is 
    # well described by the background model or not. Related to Cthr from the paper. 
    mog2.setVarThreshold(4.5**2)                  # default: 16

    # Threshold for the squared Mahalanobis distance that helps decide when a sample is close to the existing 
    # components (corresponds to Tg in the paper). If a pixel is not close to any component, it is considered
    # foreground or added as a new component. 3 sigma => Tg=3*3=9 is default. A smaller Tg value generates 
    # more components. A higher Tg value may result in a small number of components but they can grow too large. 
    # mog2.setVarThresholdGen(3**2)               # default:  9
    return mog2

# SETUP - MOG2 Background Subtractor
mog2 = create_mog2_background_subtractor()

A new object should remain static for 21.2 frames i.e. 1.77 seconds


In [19]:
# SETUP - Temporal Median Background
frame_buffer = []
frame_count = 0
skip_count = 0
temporalMedianBackground = None
 
fgmask_diff = None

if os.path.exists(file_output):
    os.remove(file_output)

# header of the output file
header = [['Frame ID', 'Number of detected objects'], ['Object ID', 'Area', 'Perimeter', 'Classification', '[Phantom]']]
with open(file_output, "a", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(header)

while(cap.isOpened()):
    ret, frame_original = cap.read()
    if not ret or frame_original is None:
        cap.release()
        print("Released Video Resource")
        break

    frame_count += 1
    skip_count += 1

    frame = cv2.cvtColor(frame_original, cv2.COLOR_BGR2GRAY)
    frame = preprocessing(frame, ProcessingType.DENOISE)

    #     ------------------------------
    # [1] |  TEMPORAL MEDIAN FILTER    |
    #     ------------------------------
    if len(frame_buffer) == 0 or skip_count == SKIP_FRAMES:
        skip_count = 0
        frame_buffer.append(frame)
        if len(frame_buffer) > MAX_HISTORY:
            frame_buffer.pop(0)

        temporalMedianBackground = np.median(frame_buffer, axis=0).astype(dtype=np.uint8)
        temporalMedianBackground_copy = temporalMedianBackground.copy()
        cv2.putText(temporalMedianBackground_copy, f"FRAME: {frame_count}/{total_frame_count}", (5, 25), font, 0.5, (0, 0, 0), 1) 
        SHOW_GUI and cv2.imshow("temporalMedianBackground", temporalMedianBackground_copy)

    #     -------------------------------
    # [2] |            MOG2             |
    #     -------------------------------
    learning_rate = compute_learning_rate(frame_count, mog2.getHistory(), dynamic=False)
    mog2_fgmask = mog2.apply(frame, learning_rate)
    mog2_fgmask_copy = mog2_fgmask.copy()
    cv2.putText(mog2_fgmask_copy, f"FRAME: {frame_count}/{total_frame_count}", (5, 25), font, 0.5, 255, 1) 
    SHOW_GUI and cv2.imshow("mog2_fgmask", mog2_fgmask_copy)

    mog2_fgmask = postprocessing(mog2_fgmask, ProcessingType.MOG)
    SHOW_GUI and cv2.imshow("mog2_fgmask postprocess", mog2_fgmask)

    if frame_count > LEARNING_PHASE:
        #     -------------------------------
        # [3] |            ROI              |
        #     -------------------------------
        found, roi = get_roi_foreground(mog2_fgmask)
        
        args = (STATIC_THRESHOLD_HYSTERESIS, STATIC_THRESHOLD)
        tmb_fgmask = background_subtraction(frame, temporalMedianBackground, False, args)
        SHOW_GUI and cv2.imshow("tmb_fgmask", tmb_fgmask)
        tmb_fgmask = postprocessing(tmb_fgmask, ProcessingType.TEMPORAL_MEDIAN)
        SHOW_GUI and cv2.imshow("tmb_fgmask postprocess", tmb_fgmask)
        
        if found:
            combined_fgmask = mog2_fgmask.copy()
            x_min, x_max, y_min, y_max = roi
            # [ DRAW ROI ]
            # frame_contours = frame.copy()
            # cv2.rectangle(frame_contours, (x_min, y_min), (x_max, y_max), 255, 2)
            # cv2.imshow("frame ROI", frame_contours)

            # mog2_fgmask_copy = mog2_fgmask.copy()
            # cv2.rectangle(mog2_fgmask_copy, (x_min, y_min), (x_max, y_max), 127, 2)
            # cv2.imshow("mog2 ROI", mog2_fgmask_copy)

            # tmb_fgmask_copy = tmb_fgmask.copy()
            # cv2.rectangle(tmb_fgmask_copy, (x_min, y_min), (x_max, y_max), 127, 2)
            # cv2.imshow("tmb ROI", tmb_fgmask_copy)

            #     -------------------------------
            # [4] |    COMBINE FGMASK IN ROI    |
            #     -------------------------------
            combined_fgmask[y_min:y_max, x_min:x_max] = cv2.bitwise_or(mog2_fgmask[y_min:y_max, x_min:x_max], tmb_fgmask[y_min:y_max, x_min:x_max])    
            combined_fgmask = postprocessing(combined_fgmask, ProcessingType.COMBINED)
            SHOW_GUI and cv2.imshow("combined_fgmask postprocessing", combined_fgmask)
            
            #     --------------------------------
            # [5] |   STATIC OBJECT DETECTION    |
            #     --------------------------------
            fgmask_diff_new = cv2.bitwise_and(tmb_fgmask, cv2.bitwise_not(combined_fgmask))
            if fgmask_diff is None:
                fgmask_diff = fgmask_diff_new
            
            #     ---------------------------------
            # [6] |   PRESERVE SEEN DIFFERENCES   |
            #     ---------------------------------
            fgmask_diff = cv2.bitwise_or(fgmask_diff, fgmask_diff_new)
            # fgmask_diff = postprocessing(fgmask_diff, ProcessingType.DIFF)
            SHOW_GUI and cv2.imshow('frame diff', fgmask_diff)
        
            complete_mask = cv2.bitwise_or(combined_fgmask, fgmask_diff)

            # fill masks
            complete_mask = process_contours(complete_mask)
            fgmask_diff = process_contours(fgmask_diff)
            combined_fgmask = process_contours(combined_fgmask)                

            #     -----------------------------------
            # [7] |  CONNECTED COMPONENTS ANALYSIS  |
            #     -----------------------------------
            output_diff = cv2.connectedComponentsWithStats(fgmask_diff, connectivity=8)
            (num_labels_diff, labels_diff, stats_diff, centroids_diff) = output_diff
            output_comb = cv2.connectedComponentsWithStats(combined_fgmask, connectivity=8)
            (num_labels_comb, labels_comb, stats_comb, centroids_comb) = output_comb

            output_complete = cv2.connectedComponentsWithStats(complete_mask, connectivity=8)
            (num_labels_complete, labels_complete, stats_complete, centroids_complete) = output_complete
            
            labeled = imshow_components(labels_diff, labels_comb)
            cv2.putText(labeled, f"FRAME: {frame_count}/{total_frame_count}", (5, 25), font, 0.5, (255, 255, 255), 1) 
            cv2.imshow("labeled", labeled)

            contours_fg, _ = cv2.findContours(complete_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            frame_original_contours = frame_original.copy()
            frame_original_filled = frame_original.copy()
            frame_original_filled = cv2.drawContours(frame_original_filled, contours_fg, -1, (0,0,255), cv2.FILLED)
            frame_original_contours = cv2.drawContours(frame_original_contours, contours_fg, -1, (0,255,0), 3)
            SHOW_GUI and cv2.imshow("complete filled", frame_original_filled)
            SHOW_GUI and cv2.imshow('complete contours', frame_original_contours)

            # results on text file
            real_labels = num_labels_complete - 1 # -1 background
            data = [[frame_count, real_labels]]
            frame_original_rects = frame_original.copy()
            for i in range(len(contours_fg)):
                area = int(cv2.contourArea(contours_fg[i]))
                perimeter = int(cv2.arcLength(contours_fg[i], True))
                if area > 5000:
                    type = 'person'
                    data.append([i+1, area, perimeter, type])
                else:
                    if area < 100:
                        continue

                    type = 'other'
                    
                    phantom = find_phantoms(contours_fg[i], frame, temporalMedianBackground, fgmask_diff)
                    if phantom == -1:
                        continue
                            
                    data.append([i+1, area, perimeter, type, not(phantom)])
            
            SHOW_GUI and cv2.imshow("RECTS", frame_original_rects)

            with open(file_output, "a", newline="") as file:
                writer = csv.writer(file)
                writer.writerows(data)
                        

    cmd = cv2.waitKey(0)    
    if cmd == ord("q"):
        break
    if cmd == ord("n"):
        continue

cap.release()
cv2.destroyAllWindows()


Released Video Resource
